# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [2]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [3]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:21<00:00, 21.71s/it]


In [5]:
len(deals)

10

In [14]:
print(list(deals))

[<Fremo X300 276Wh Portable Power Station for $170 + free shipping>, <Samsung Certified Re-Newed Smartphones for $350 minimum trade-in credit + free shipping>, <Samsung Crystal UHD DU7200 TVs: Up to $200 off>, <KEH Camera End of Year Sale: Up to 20% off + extra 10% to 20% off + free shipping w/ $75>, <NanoPhone 4G Dual-SIM Android Smartphone for $90 + free shipping>, <iScanner App for iPhone for $28>, <Energizer Last Minute Deals at Walmart From $3 + free shipping w/ $35>, <RedPocket Unlimited Prepaid Plan for $140 + free shipping>, <Tech Gifts at Walmart Under $25 + free shipping w/ $35>, <Infimobile 5GB 12-Month Plan for $75>]


In [15]:
deals[0].describe()

"Title: Fremo X300 276Wh Portable Power Station for $170 + free shipping\nDetails: As part of today's daily deals, bag this one at the lowest price we've ever seen on it. Buy Now at Best Buy\nFeatures: EVE lithium-ion battery pure sine wave inverter  built-in Battery Management System supports USB-C PD 100W charging and standard adapter charging Model: X300\nURL: https://www.dealnews.com/products/Fremo/Fremo-X300-276-Wh-Portable-Power-Station/393098.html?iref=rss-c142"

In [8]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [25]:
print(system_prompt)

You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentio

In [9]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [24]:
print(user_prompt)

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Fremo X300 276Wh Portable Power Station for $170 + free shipping
Details: As part of today's daily deals, bag this one at the lowest price we've ever seen on it. Buy Now at Best Buy
Features: EVE lithium-ion battery pure sine wave inverter  built-in Battery Management System supports USB-C PD 100W charging and standard adapter charging Model: X300
URL: https://www.d

In [16]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [17]:
result = get_recommendations()

In [22]:
type(result)

agents.deals.DealSelection

In [29]:
result

DealSelection(deals=[Deal(product_description='The Fremo X300 Portable Power Station is a compact, lightweight solution for your power needs on-the-go. With a 276Wh capacity, it features advanced EVE lithium-ion battery technology and a pure sine wave inverter, ensuring safe and stable energy output for various devices. The built-in Battery Management System enhances performance and safety, while USB-C PD 100W charging allows for fast charging of compatible devices. Perfect for camping, emergencies, or as a mobile power source, this power station is a must-have for outdoor enthusiasts.', price=170.0, url='https://www.dealnews.com/products/Fremo/Fremo-X300-276-Wh-Portable-Power-Station/393098.html?iref=rss-c142'), Deal(product_description='The NanoPhone 4G is an innovative, credit card-sized Android smartphone designed for simplicity and convenience. It features a vibrant 3-inch IPS display and includes essential functionalities with a 5MP rear camera and a 2MP front camera for capturin

In [26]:
len(result.deals)

5

In [28]:
print(result.deals[0])

product_description='The Fremo X300 Portable Power Station is a compact, lightweight solution for your power needs on-the-go. With a 276Wh capacity, it features advanced EVE lithium-ion battery technology and a pure sine wave inverter, ensuring safe and stable energy output for various devices. The built-in Battery Management System enhances performance and safety, while USB-C PD 100W charging allows for fast charging of compatible devices. Perfect for camping, emergencies, or as a mobile power source, this power station is a must-have for outdoor enthusiasts.' price=170.0 url='https://www.dealnews.com/products/Fremo/Fremo-X300-276-Wh-Portable-Power-Station/393098.html?iref=rss-c142'


In [ ]:
from agents.scanner_agent import ScannerAgent

In [ ]:
agent = ScannerAgent()
result = agent.scan()

In [ ]:
result